# Sentiment Analysis with an RNN

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
with open('../sentiment_network/reviews.txt', 'r') as f:
    reviews = f.read()
with open('../sentiment_network/labels.txt', 'r') as f:
    labels = f.read()

In [ ]:
reviews[:2000]

In [ ]:
labels[:100]

## Data preprocessing

In [3]:
from string import punctuation
all_text = ''.join([c for c in reviews if c not in punctuation])
reviews = all_text.split('\n')

all_text = ' '.join(reviews)
words = all_text.split()

In [ ]:
all_text[:2000]

In [ ]:
words[:100]

#### Encoding the words

In [4]:
from collections import Counter
counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)

# Create dictionary that maps vocab words to integers
vocab_to_int = {word: i for i, word in enumerate(vocab, 1)}

# Convert the reviews to integers, same shape as reviews list, but with integers
reviews_ints = []
for each in reviews:
    reviews_ints.append([vocab_to_int[word] for word in each.split()])

#### Encoding the labels

In [5]:
# Convert labels to 1s and 0s for 'positive' and 'negative'
labels = labels.split('\n')
labels = np.array([1 if each == 'positive' else 0 for each in labels])

In [6]:
from collections import Counter
review_lens = Counter([len(x) for x in reviews_ints])
print("Zero-length reviews: {}".format(review_lens[0]))
print("Maximum review length: {}".format(max(review_lens)))

Zero-length reviews: 1
Maximum review length: 2514


In [8]:
# Filter out that review with 0 length
reviews_ints = [each for each in reviews_ints if len(each) > 0]

In [9]:
# Each row should be 200 elements long
seq_len = 200
features =  np.zeros((len(reviews), seq_len), dtype=int)

for i, row in enumerate(reviews_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [ ]:
features[:10,:100]

#### Training, Validation, Test

In [12]:
split_frac = 0.8
idx1 = int(len(features)*0.8)
idx2 = int(len(features)*0.9)

train_x, val_x, test_x = features[:idx1], features[idx1:idx2], features[idx2:]
train_y, val_y, test_y = labels[:idx1], labels[idx1:idx2], labels[idx2:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

			Feature Shapes:
Train set: 		(20000, 200) 
Validation set: 	(2500, 200) 
Test set: 		(2501, 200)


## Build the graph

#### Define the hyperparameters.

* `lstm_size`: Number of units in the hidden layers in the LSTM cells. Usually larger is better performance wise. Common values are 128, 256, 512, etc.
* `lstm_layers`: Number of LSTM layers in the network. 
* `batch_size`: The number of reviews to feed the network in one training pass. Typically this should be set as high as you can go without running out of memory.
* `learning_rate`: Learning rate

In [15]:
lstm_size = 256
lstm_layers = 1
batch_size = 512
learning_rate = 0.001

#### Initialize the graph

In [13]:
n_words = len(vocab)

# Create the graph object
graph = tf.Graph()
# Add nodes to the graph
with graph.as_default():
    inputs_ = tf.placeholder(tf.int32, [None, None], name='inputs')
    labels_ = tf.placeholder(tf.int32, [None, None], name='labels')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

#### Embedding

In [14]:
# Size of the embedding vectors (number of units in the embedding layer)
embed_size = 300 

with graph.as_default():
    embedding = tf.Variable(tf.random_uniform((n_words, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs_)

#### LSTM cell

In [16]:
with graph.as_default():
    # Basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([drop] * lstm_layers)    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)

#### RNN forward pass

In [17]:
with graph.as_default():
    outputs, final_state = tf.nn.dynamic_rnn(cell, embed, initial_state=initial_state)

#### Output

In [18]:
with graph.as_default():
    predictions = tf.contrib.layers.fully_connected(outputs[:, -1], 1, activation_fn=tf.sigmoid)
    cost = tf.losses.mean_squared_error(labels_, predictions)
    
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

#### Validation accuracy

In [19]:
with graph.as_default():
    correct_pred = tf.equal(tf.cast(tf.round(predictions), tf.int32), labels_)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#### Batching

This is a simple function for returning batches from our data. First it removes data such that we only have full batches. Then it iterates through the `x` and `y` arrays and returns slices out of those arrays with size `[batch_size]`.

In [20]:
def get_batches(x, y, batch_size=100):
    
    n_batches = len(x)//batch_size
    x, y = x[:n_batches*batch_size], y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        yield x[ii:ii+batch_size], y[ii:ii+batch_size]

## Training

In [21]:
epochs = 10

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
    for e in range(epochs):
        state = sess.run(initial_state)
        
        for ii, (x, y) in enumerate(get_batches(train_x, train_y, batch_size), 1):
            feed = {inputs_: x,
                    labels_: y[:, None],
                    keep_prob: 0.5,
                    initial_state: state}
            loss, state, _ = sess.run([cost, final_state, optimizer], feed_dict=feed)
            
            if iteration%5==0:
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Train loss: {:.3f}".format(loss))

            if iteration%25==0:
                val_acc = []
                val_state = sess.run(cell.zero_state(batch_size, tf.float32))
                for x, y in get_batches(val_x, val_y, batch_size):
                    feed = {inputs_: x,
                            labels_: y[:, None],
                            keep_prob: 1,
                            initial_state: val_state}
                    batch_acc, val_state = sess.run([accuracy, final_state], feed_dict=feed)
                    val_acc.append(batch_acc)
                print("Val acc: {:.3f}".format(np.mean(val_acc)))
            iteration +=1
    saver.save(sess, "checkpoints/sentiment.ckpt")

Epoch: 0/10 Iteration: 5 Train loss: 0.243
Epoch: 0/10 Iteration: 10 Train loss: 0.242
Epoch: 0/10 Iteration: 15 Train loss: 0.216
Epoch: 0/10 Iteration: 20 Train loss: 0.204
Epoch: 0/10 Iteration: 25 Train loss: 0.397
Val acc: 0.560
Epoch: 0/10 Iteration: 30 Train loss: 0.239
Epoch: 0/10 Iteration: 35 Train loss: 0.240
Epoch: 1/10 Iteration: 40 Train loss: 0.219
Epoch: 1/10 Iteration: 45 Train loss: 0.216
Epoch: 1/10 Iteration: 50 Train loss: 0.212
Val acc: 0.631
Epoch: 1/10 Iteration: 55 Train loss: 0.220
Epoch: 1/10 Iteration: 60 Train loss: 0.182
Epoch: 1/10 Iteration: 65 Train loss: 0.150
Epoch: 1/10 Iteration: 70 Train loss: 0.176
Epoch: 1/10 Iteration: 75 Train loss: 0.175
Val acc: 0.755
Epoch: 2/10 Iteration: 80 Train loss: 0.149
Epoch: 2/10 Iteration: 85 Train loss: 0.143
Epoch: 2/10 Iteration: 90 Train loss: 0.167
Epoch: 2/10 Iteration: 95 Train loss: 0.138
Epoch: 2/10 Iteration: 100 Train loss: 0.153
Val acc: 0.740
Epoch: 2/10 Iteration: 105 Train loss: 0.187
Epoch: 2/10 Ite

## Testing

In [ ]:
test_acc = []
with tf.Session(graph=graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('/output/checkpoints'))
    test_state = sess.run(cell.zero_state(batch_size, tf.float32))
    for ii, (x, y) in enumerate(get_batches(test_x, test_y, batch_size), 1):
        feed = {inputs_: x,
                labels_: y[:, None],
                keep_prob: 1,
                initial_state: test_state}
        batch_acc, test_state = sess.run([accuracy, final_state], feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.3f}".format(np.mean(test_acc)))